In [236]:
import numpy

def doublerangestr(n):
    #returns list of stings [00,01,02,...,n-1]
    
    #needed for some methods on polynomial rings
    #where no variable is allowed to be prefix of another
    #i.e. x1 and x10
    
    list = []
    for i in range(n):
        if i<10:
            list.append('0'+str(i))
        else:
            list.append(str(i))
    return list

def create_polytope(A,b):
    #returns polytope given by x>=0, Ax=b
    pos = [[-b[i][0]]+M[i] for i in range(d)]
    neg = [[b[i][0]]+map(lambda x: -x, M[i]) for  i in range(d)]
    units = []
    for i in range(n):
        zeros = [0]*(n)
        zeros[i]=1
        units.append([0]+zeros)
    ineqs = pos+neg+units
    return Polyhedron(ieqs = ineqs)

def support(vector):
    #returns the support of a given vector
    v = list(vector)
    return [i for i in range(len(v)) if v[i]>0]

def compute_v(A, sigma):
    #returns for given circuit sigma the respective v_sigma 
    #havving support on sigma and A*v_sigma = b
    A_sigma = matrix(ZZ, [A.column(i) for i in sigma]).transpose()
    v = matrix(SR, [var('v'+str(i)) for i in sigma]).transpose()
    eqn = [(A_sigma * v)[i][0] == var('b'+str(i)) for i in range(len(sigma))]
    S = solve(eqn, [var('v'+str(i)) for i in sigma])
    return [eq.rhs() for eq in S[0]]

def basic_circuit(sigma,i):
    #returns the unique vector in kerA having ith coordinate = 1
    #and support on sigma and i
    v = [0]*n
    v[i]=1
    for j in sigma:
        v[j] = var('v'+str(j))
    v = matrix(SR, v).transpose()
    eqn = [(A * v)[i][0] == 0 for i in range(d)]
    S = solve(eqn, [var('v'+str(i)) for i in sigma])
    v = [el[0] for el in v]
    for j in range(len(sigma)):
        v[sigma[j]]=S[0][j].rhs()
    return v

## Input

In [237]:
M = [
    [1,0,0,1,0,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0,0,1,0,0],
    [0,0,0,0,1,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,0,1,0,0,1],
    [1,0,0,0,1,0,1,0,0,0,0,0],
    [1,0,0,0,0,1,0,0,0,1,0,0],
    [0,1,0,0,0,0,0,0,1,1,0,0],
    [0,0,0,0,1,0,0,0,1,0,1,0]
]


M = [
    [ 1, 1, 1, 0, 0, 0],
    [-1, 0, 0, 1, 1, 0],
    [ 0,-1, 0,-1, 0, 1]
]



b = Matrix(ZZ, [1,3,-2]).transpose()
d = len(M)
n = len(M[0])

A = Matrix(ZZ, M)
#test if A is totally unimodualar
for k in range(len(M)+1):
    for x in A.minors(k):
        if abs(x)>1:
            print "not unimodular"

## Algorithm 2: Computing the generating function

In [238]:
##Step 1: Compute Circuits of A
matroid = Matroid(A)
circ = sorted([sorted(C) for C in matroid.circuits()])

def cone_functions(A,b):
    ##Step 2: List all indices of vertices of P_b
    P_b = create_polytope(A,b)
    #f = 0
    cones = []
    for vertex in P_b.vertices():
        sigma = support(vertex)
        ##Step 3: Compute RHS of (7)
            #part one: x^(v_sigma)
        v_sigma = compute_v(A,sigma)
        first_part = numpy.prod([var('x'+str(sigma[i]))^v_sigma[i] for i in range(len(sigma))])
            #part two: (1-x^c)^(-1)
        sigma_complement = [i for i in range(n) if i not in sigma]
        second_part = []
        for i in sigma_complement:
            c = basic_circuit(sigma,i)
            inner = numpy.prod([ var('x'+str(k))^c[k] for k in range(n)])
            second_part.append( (1-inner)^(-1) )
        f_cone = first_part*numpy.prod(second_part)
        cones.append(f_cone)
        #f = f + f_cone
        print 'index set: '+str(sigma)
        print 'rational function: '
        print f_cone
        print ' '
    return cones

In [239]:
cones = cone_functions(A,b)
f = sum(cones)
print 'generating function:'
print f

index set: [0, 3, 4]
rational function: 
-x0^b0*x4^(b0 + b1 + b2)/(x3^b2*(x3*x5/x4 - 1)*(x1/(x0*x3) - 1)*(x2/(x0*x4) - 1))
 
index set: [0, 3, 5]
rational function: 
-x0^b0*x3^(b0 + b1)*x5^(b0 + b1 + b2)/((x1/(x0*x3) - 1)*(x4/(x3*x5) - 1)*(x2/(x0*x3*x5) - 1))
 
index set: [2, 3, 4]
rational function: 
-x2^b0*x4^(b1 + b2)/(x3^b2*(x0*x4/x2 - 1)*(x3*x5/x4 - 1)*(x1*x4/(x2*x3) - 1))
 
index set: [1, 3, 5]
rational function: 
-x1^b0*x3^b1*x5^(b0 + b1 + b2)/((x0*x3/x1 - 1)*(x2/(x1*x5) - 1)*(x4/(x3*x5) - 1))
 
index set: [2, 3, 5]
rational function: 
-x2^b0*x3^b1*x5^(b1 + b2)/((x0*x3*x5/x2 - 1)*(x1*x5/x2 - 1)*(x4/(x3*x5) - 1))
 
index set: [1, 3, 4]
rational function: 
-x1^b0*x3^(-b0 - b2)*x4^(b0 + b1 + b2)/((x0*x3/x1 - 1)*(x3*x5/x4 - 1)*(x2*x3/(x1*x4) - 1))
 
generating function:
-x1^b0*x3^(-b0 - b2)*x4^(b0 + b1 + b2)/((x0*x3/x1 - 1)*(x3*x5/x4 - 1)*(x2*x3/(x1*x4) - 1)) - x2^b0*x3^b1*x5^(b1 + b2)/((x0*x3*x5/x2 - 1)*(x1*x5/x2 - 1)*(x4/(x3*x5) - 1)) - x1^b0*x3^b1*x5^(b0 + b1 + b2)/((x0*x3/x1 - 1

## Algorithm 3: Evaluating at (1,...,1)

In [240]:
P_b = create_polytope(A,b)
basis = support(P_b.vertices()[0])
basis_complement = [k for k in range(n) if k not in basis]
t = var('t')

for k in range(len(cones)):
    g = cones[k]
    #Step 1: Eliminate variables
    for i in basis:
        g = g.subs({var('x'+str(i)):1})
    #Step 2: Substitution
    for j in basis_complement:
        g = g.subs({var('x'+str(j)):1-j*t})
    cones[k]=g
    #print g.expand()
    #Step 2: Represent g as numerator/denominator
    
#Step 3: Compute least common multiple q of denominators
#Step 3: Compute p such that p = q*f
#Step 4: l'hopital
#step 5: return (p/g)(0)

